#Importing Package

In [34]:
pip install turicreate

In [35]:
import re
import turicreate as tc
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import string
import random

In [36]:
import gc
import time
import seaborn as sns
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

#Working on UserId and ProdID

In [37]:
fashion = tc.SFrame.read_csv('/content/drive/My Drive/AMAZON_FASHION.csv',header = None)

Finished parsing file /content/drive/My Drive/AMAZON_FASHION.csv

Parsing completed. Parsed 100 lines in 2.05939 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,float,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /content/drive/My Drive/AMAZON_FASHION.csv

Parsing completed. Parsed 883636 lines in 1.23951 secs.

In [38]:
fashion.rename({'X1':'ProdID','X2':'UserID','X3':'Rating','X4':'Timestamp'},inplace=True)
fashion.remove_column('Timestamp',inplace=True)
fashion

ProdID,UserID,Rating
7106116521,A1D4G1SNUZWQOT,5.0
7106116521,A3DDWDH9PX2YX2,2.0
7106116521,A2MWC41EW7XL15,4.0
7106116521,A2UH2QQ275NV45,2.0
7106116521,A89F3LQADZBS5,3.0
7106116521,A29HLOUW0NS0EH,5.0
7106116521,A7QS961ROI6E0,4.0
B00007GDFV,A1BB77SEBQT8VX,3.0
B00007GDFV,AHWOW7D1ABO9C,3.0
B00007GDFV,AKS3GULZE0HFC,3.0


Rating associate with each Products


In [39]:
fashion.groupby(key_column_names='ProdID',operations={'mean_rating': tc.aggregate.MEAN('Rating'),'std_rating': tc.aggregate.STD('Rating')})

ProdID,mean_rating,std_rating
B00LVYEEPG,4.0,0.0
B0010TJ9FC,4.5,0.5
B00W0Z402M,5.0,0.0
B01CTQJ2ZQ,3.0,2.0
B00ZJD3FTG,5.0,0.0
B015IRGGF2,1.5,0.5
B005HBIYIW,3.0,0.0
B01DVB0TO6,3.0,2.0
B00NTJDUI8,4.0,0.0
B017XEPI6G,4.5,0.5000000000000001


 mean and standard deviation of ratings per user.

In [40]:
fashion.groupby(key_column_names='UserID',operations={'mean_rating': tc.aggregate.MEAN('Rating'),'std_rating': tc.aggregate.STD('Rating')})

UserID,mean_rating,std_rating
A1KTRP6I0YEPF4,5.0,0.0
A3FUPR2H79C8RL,5.0,0.0
A3A9LY4MT89JIY,5.0,0.0
A1FA177XLVW2YC,5.0,0.0
AU52XXO3CA4QE,5.0,0.0
A14WRMQHG2V84L,5.0,0.0
A1B9W1YG8PH91L,3.0,0.0
A2JFIL3PTLSM4B,5.0,0.0
AKLLJXYE1PNUH,2.0,0.0
A3OS63ZWKSA33Y,3.4285714285714284,1.178030178747903


Max rated movie by each User

In [41]:
fashion.groupby(key_column_names='UserID', operations={('max_rated_movie'): tc.aggregate.ARGMAX('Rating','ProdID')})


UserID,max_rated_movie
A1KTRP6I0YEPF4,B00FN90O8G
A3FUPR2H79C8RL,B00M9O0FBE
A3A9LY4MT89JIY,B00XO8RUGC
A1FA177XLVW2YC,B01F4RWN9E
AU52XXO3CA4QE,B002FVPQWY
A14WRMQHG2V84L,B015BNKNWU
A1B9W1YG8PH91L,B01401RV4M
A2JFIL3PTLSM4B,B00NPORRFE
AKLLJXYE1PNUH,B0183JQHCO
A3OS63ZWKSA33Y,B009ERSRDM


#Working on feature related to ProdID

In [42]:
fashion_meta = tc.SFrame.read_json('/content/drive/My Drive/meta_AMAZON_FASHION.json',orient='lines')

Finished parsing file /content/drive/My Drive/meta_AMAZON_FASHION.json

Parsing completed. Parsed 100 lines in 1.33073 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[dict]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 9221 lines. Lines per second: 3574.36

Read 54224 lines. Lines per second: 6175.82

Read 99123 lines. Lines per second: 6670.88

Read 143745 lines. Lines per second: 6837.96

Finished parsing file /content/drive/My Drive/meta_AMAZON_FASHION.json

Parsing completed. Parsed 186627 lines in 25.8222 secs.

In [43]:
fashion_meta.remove_columns(['tech1','price','also_buy','rank','also_view','description','feature','image','tech2','category','similar_item','details','main_cat','date','fit'],inplace = True)
fashion_meta.rename({'asin':'ProdID'},inplace=True)

ProdID,brand,title
630456984X,,Dante's Peak - Laserdisc
7106116521,,Milliongadgets(TM)Earring Safety Backs For ...
8037200124,Envirosax,Envirosax Kids SeriesJessie &amp; Lulu ...
8037200221,Envirosax,Envirosax GreengrocerSeries Bag 7 Guava ...
8279996567,Gifts by Lulee,Blessed by Pope BenedettoXVI Our Lady of Guada ...
9239282785,Tideclothes,Tideclothes ALAGIRLSStrapless Beading ...
9239281533,Tideclothes,ALAGIRLS StraplessBeading Homecoming Prom ...
9269808971,,Syma S107C 3channelCoaxial Mini Spy Cam ...
9654263246,Roar Carbon,"X. L. Carbon Fiber MoneyClip, made in the USA ..."
B00004T3SN,Shimmer Anne Shine,Shimmer Anne Shine ClipOn Costume/Halloween Cat ...


In [44]:
def Filtering_brand(s):
  if s.lower() == '' or s.lower() == 'unknown':
    return 'Unknown'
  return s
def remove_punctuation(text):
    translator = text.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    return text

In [45]:
fashion_meta['brand'] = fashion_meta['brand'].apply(Filtering_brand)

In [46]:
Title_without_punctuation = fashion_meta['title'].apply(remove_punctuation)
fashion_meta['title_tf_idf'] = tc.text_analytics.tf_idf(Title_without_punctuation)


In [47]:
fashion_meta

ProdID,brand,title,title_tf_idf
630456984X,Unknown,Dante's Peak - Laserdisc,"{'laserdisc':11.038254962793788, ..."
7106116521,Unknown,Milliongadgets(TM)Earring Safety Backs For ...,"{'earrings':3.987554407826554, ..."
8037200124,Envirosax,Envirosax Kids SeriesJessie &amp; Lulu ...,"{'lulu':10.190957102406585, ..."
8037200221,Envirosax,Envirosax GreengrocerSeries Bag 7 Guava ...,"{'guava':10.190957102406585, ..."
8279996567,Gifts by Lulee,Blessed by Pope BenedettoXVI Our Lady of Guada ...,"{'rosas':9.939642674125679, ..."
9239282785,Tideclothes,Tideclothes ALAGIRLSStrapless Beading ...,"{'white':2.7337600648302223, ' ..."
9239281533,Tideclothes,ALAGIRLS StraplessBeading Homecoming Prom ...,"{'alagirls':9.141134977907907, ..."
9269808971,Unknown,Syma S107C 3channelCoaxial Mini Spy Cam ...,"{'included':8.18562353288047, 'ca ..."
9654263246,Roar Carbon,"X. L. Carbon Fiber MoneyClip, made in the USA ...","{'usa':5.145690364340688, 'in': ..."
B00004T3SN,Shimmer Anne Shine,Shimmer Anne Shine ClipOn Costume/Halloween Cat ...,"{'cat':5.088480842740015, 'on': ..."


In [48]:
fashion_meta.remove_column('title',inplace=True)

ProdID,brand,title_tf_idf
630456984X,Unknown,"{'laserdisc':11.038254962793788, ..."
7106116521,Unknown,"{'earrings':3.987554407826554, ..."
8037200124,Envirosax,"{'lulu':10.190957102406585, ..."
8037200221,Envirosax,"{'guava':10.190957102406585, ..."
8279996567,Gifts by Lulee,"{'rosas':9.939642674125679, ..."
9239282785,Tideclothes,"{'white':2.7337600648302223, ' ..."
9239281533,Tideclothes,"{'alagirls':9.141134977907907, ..."
9269808971,Unknown,"{'included':8.18562353288047, 'ca ..."
9654263246,Roar Carbon,"{'usa':5.145690364340688, 'in': ..."
B00004T3SN,Shimmer Anne Shine,"{'cat':5.088480842740015, 'on': ..."


In [49]:
Difference_Prod_in_fashion_fashion_meta = ["6041002984",
"1942705034",
"1291691480",
"5378828716",
"0764443682",
"3293015344",
"1940967805",
"B00B975LHK",
"1940280001",
"1940735033",
"B01HGBTH8Q",
"B01D8DOITO",
"B01A80WKEA"]

In [50]:
def change(ProdID):
  if ProdID not in Difference_Prod_in_fashion_fashion_meta:
    return ProdID
  return None



In [51]:
fashion['ProdID'] = fashion['ProdID'].apply(change)
fashion = fashion[fashion['ProdID'] != None]

#Split Dataset

In [52]:
fashion

ProdID,UserID,Rating
7106116521,A1D4G1SNUZWQOT,5.0
7106116521,A3DDWDH9PX2YX2,2.0
7106116521,A2MWC41EW7XL15,4.0
7106116521,A2UH2QQ275NV45,2.0
7106116521,A89F3LQADZBS5,3.0
7106116521,A29HLOUW0NS0EH,5.0
7106116521,A7QS961ROI6E0,4.0
B00007GDFV,A1BB77SEBQT8VX,3.0
B00007GDFV,AHWOW7D1ABO9C,3.0
B00007GDFV,AKS3GULZE0HFC,3.0


In [53]:
training_data, validation_data = tc.recommender.util.random_split_by_user(fashion,user_id='UserID',item_id='ProdID',random_seed= 8)

#Model Training

In [66]:
model_1 = tc.recommender.factorization_recommender.create(training_data,user_id = 'UserID',item_id = 'ProdID',target= 'Rating',max_iterations=200,
                                                          regularization=1e-20,solver = 'adagrad',linear_regularization = 1e-10)

Preparing data set.

Data has 883356 observations with 749017 users and 186144 items.

Data prepared in: 2.0857s

Training factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Convergence on objective within bounds.

Computing final objective value and training RMSE.

Final objective value: 0.000406663

Final training RMSE: 0.0199812

In [67]:
model_1.evaluate(validation_data)


Precision and recall summary statistics by cutoff
+--------+----------------+-------------+
| cutoff | mean_precision | mean_recall |
+--------+----------------+-------------+
|   1    |      0.0       |     0.0     |
|   2    |      0.0       |     0.0     |
|   3    |      0.0       |     0.0     |
|   4    |      0.0       |     0.0     |
|   5    |      0.0       |     0.0     |
|   6    |      0.0       |     0.0     |
|   7    |      0.0       |     0.0     |
|   8    |      0.0       |     0.0     |
|   9    |      0.0       |     0.0     |
|   10   |      0.0       |     0.0     |
+--------+----------------+-------------+
[10 rows x 3 columns]


Overall RMSE: 1.3679107104377068

Per User RMSE (best)
+----------------+------------------------+-------+
|     UserID     |          rmse          | count |
+----------------+------------------------+-------+
| A12KIRTJCYKCFZ | 0.00012744679643539314 |   1   |
+----------------+------------------------+-------+
[1 rows x 3 columns]



{'precision_recall_by_user': Columns:
 	UserID	str
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 4590
 
 Data:
 +----------------+--------+-----------+--------+-------+
 |     UserID     | cutoff | precision | recall | count |
 +----------------+--------+-----------+--------+-------+
 | A1MY5D77UNWLI8 |   1    |    0.0    |  0.0   |   1   |
 | A1MY5D77UNWLI8 |   2    |    0.0    |  0.0   |   1   |
 | A1MY5D77UNWLI8 |   3    |    0.0    |  0.0   |   1   |
 | A1MY5D77UNWLI8 |   4    |    0.0    |  0.0   |   1   |
 | A1MY5D77UNWLI8 |   5    |    0.0    |  0.0   |   1   |
 | A1MY5D77UNWLI8 |   6    |    0.0    |  0.0   |   1   |
 | A1MY5D77UNWLI8 |   7    |    0.0    |  0.0   |   1   |
 | A1MY5D77UNWLI8 |   8    |    0.0    |  0.0   |   1   |
 | A1MY5D77UNWLI8 |   9    |    0.0    |  0.0   |   1   |
 | A1MY5D77UNWLI8 |   10   |    0.0    |  0.0   |   1   |
 +----------------+--------+-----------+--------+-------+
 [4590 rows x 5 columns]
 Note: Only the head of the SFr